# Mandelbrot set in Tensorflow: unrolling iteration steps as dataflow graph

In [1]:
# imports for simulation
import tensorflow as tf
import numpy as np

# imports for visualization
import PIL.Image
from io import BytesIO
from IPython.display import clear_output, Image, display

# imports for interaction 
from ipywidgets import interact, FloatSlider
import time

sess = tf.InteractiveSession()

In [2]:
class Timer:
    def __init__(self):
        self.perf_start = time.perf_counter()
        self.proc_start = time.process_time()
    def restart(self):
        self.perf_start = time.perf_counter()
        self.proc_start = time.process_time()
    def report(self):
        return str(round((time.perf_counter()-self.perf_start),3))
    def report_restart(self):
        result = self.report()
        self.restart()
        return result

In [3]:
def DisplayFractal(a, fmt='jpeg'):
  """given a 2D array of escape-time iteration counts, 
     render as an image"""
  a_cyclic = (6.28*a/20.0).reshape(list(a.shape)+[1])
  img = np.concatenate([10+20*np.cos(a_cyclic),
                        30+50*np.sin(a_cyclic),
                        155-80*np.cos(a_cyclic)], 2)
  img[a==a.max()] = 0
  a = img
  a = np.uint8(np.clip(a, 0, 255))
  f = BytesIO()
  PIL.Image.fromarray(a).save(f, fmt)
  clear_output(wait = True)
  display(Image(data=f.getvalue()))

In [4]:
tim1 = Timer()
maxiter = 200
# numpy setup, result Z is 1D array of complex numbers
nY, nX = np.mgrid[-2.0:2.0:0.005, -2.0:2.0:0.005] 
nZ = nX+1j*nY
print(type(nZ))
print("numpy: " + tim1.report_restart())

sess.close()
tf.reset_default_graph()
sess = tf.InteractiveSession()
print("sess startup:", tim1.report_restart())

tzoom = tf.placeholder(np.complex64, shape=())
toffset = tf.placeholder(np.complex64, shape=())
C = tf.Variable(nZ.astype(np.complex64))
C = C/tzoom + toffset
Z = C
M = tf.Variable(tf.zeros_like(C, tf.float32))

for i in range(maxiter):
    Z = Z*Z + C
    not_diverged = (tf.abs(Z) <= 4)
    M = M + tf.cast(not_diverged, tf.float32)
    
# recoord = tf.Variable(C)
# rezero = tf.Variable(tf.zeros_like(C,tf.float32))

print("graph setup:", tim1.report_restart())
tf.global_variables_initializer().run()
print("graph init:", tim1.report_restart())
#C.eval(feed_dict = {tzoom:(1.0+1.0j), toffset:(-0.5+0.0j)})
# DisplayFractal(M.eval(feed_dict = {tzoom:(1.0+1.0j), toffset:(-0.5+0.0j)}))
# print("graph compute/display:", tim1.report_restart())



<class 'numpy.ndarray'>
numpy: 0.009
sess startup: 0.001
graph setup: 1.573
graph init: 0.311


In [5]:
def view_changed(zoomer, xoff, yoff): 
    vtime = Timer()
    zscaling = zoomer + 1j*zoomer
    zoffset = xoff + 1j*yoff
    result = M.eval(feed_dict = {tzoom: zscaling, toffset: zoffset})
    compute_time = vtime.report_restart()
    DisplayFractal(result)
    display_time = vtime.report_restart()
    print("compute time: " + compute_time)
    print("display time: " + display_time)
    
interact(view_changed, 
         zoomer = FloatSlider(min=1,max=50,step=0.1,value=1, continuous_update=True), 
         xoff = FloatSlider(min=-2, max=2, step=0.05, value=-0.5, continuous_update=True), 
         yoff = FloatSlider(min=-2, max=2, step=0.05, value=0.0, continuous_update=True))

interactive(children=(FloatSlider(value=1.0, description='zoomer', max=50.0, min=1.0), FloatSlider(value=-0.5, description='xoff', max=2.0, min=-2.0, step=0.05), FloatSlider(value=0.0, description='yoff', max=2.0, min=-2.0, step=0.05), Output()), _dom_classes=('widget-interact',))

<function __main__.view_changed>

In [6]:
# sess.close()